In [6]:
import pandas as pd
import json

# Step 1: Load the JSON File
file_path = 'data/result.json'  # Replace with your file path
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Step 2: Convert Messages to a DataFrame
if 'messages' in json_data:
    messages = json_data['messages']
    df = pd.DataFrame(messages)
    # Display the first few rows of the dataset
    print("DataFrame created successfully!")
    display(df.head())
else:
    print("No 'messages' field found in the JSON file.")

# Step 3: Analyze the Dataset (Optional Example)
# Check the columns in the dataset
print("Columns in the dataset:", df.columns)

DataFrame created successfully!


,id,type,date,date_unixtime,actor,actor_id,action,title,text,text_entities,...,message_id,members,boosts,live_location_period_seconds,contact_information,contact_vcard,inline_bot_buttons,new_title,new_icon_emoji_id,reply_to_peer_id
0,1,service,2023-11-19T13:07:31,1700399251,Буховна туса,channel2138962127,migrate_from_group,Буховна туса,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,message,2023-11-19T15:12:12,1700406732,NaN,NaN,NaN,NaN,А Далі на 5? Бо на скріні 4.30…,"[{'type': 'plain', 'text': 'А Далі на 5? Бо на...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,message,2023-11-19T15:17:37,1700407057,NaN,NaN,NaN,NaN,До речі так,"[{'type': 'plain', 'text': 'До речі так'}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,message,2023-11-19T15:17:51,1700407071,NaN,NaN,NaN,NaN,"[{'type': 'mention', 'text': '@atoptunenko'}, ...","[{'type': 'mention', 'text': '@atoptunenko'}, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,message,2023-11-19T15:18:15,1700407095,NaN,NaN,NaN,NaN,Зараз,"[{'type': 'plain', 'text': 'Зараз'}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Columns in the dataset: Index(['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action',
       'title', 'text', 'text_entities', 'from', 'from_id',
       'reply_to_message_id', 'location_information', 'edited',
       'edited_unixtime', 'photo', 'width', 'height', 'reactions', 'file',
       'file_name', 'media_type', 'mime_type', 'duration_seconds', 'thumbnail',
       'poll', 'sticker_emoji', 'forwarded_from', 'message_id', 'members',
       'boosts', 'live_location_period_seconds', 'contact_information',
       'contact_vcard', 'inline_bot_buttons', 'new_title', 'new_icon_emoji_id',
       'reply_to_peer_id'],
      dtype='object')


In [7]:
# Filter messages by the 'from' field
filtered_messages_with_reactions = df[
    (df['from'] == "Pavlo Shulin") & df['reactions'].notna()
]

# Display the filtered DataFrame
print(f"Total messages from Pavlo Shulin: {len(filtered_messages)}")
display(filtered_messages)

NameError: name 'filtered_messages' is not defined

In [8]:
from collections import defaultdict

# Initialize a dictionary to hold user reaction counts
user_reaction_counts = defaultdict(int)

# Step 1: Iterate through the messages and extract reactions
for _, row in filtered_messages_with_reactions.iterrows():
    reactions = row.get('reactions')
    if isinstance(reactions, list):  # Ensure reactions exist and are a list
        for reaction in reactions:
            if 'recent' in reaction and isinstance(reaction['recent'], list):
                for recent_reaction in reaction['recent']:
                    user = recent_reaction.get('from')  # Extract 'from' field
                    count = reaction.get('count', 0)   # Extract 'count'
                    if user:  # Ensure the user field exists
                        user_reaction_counts[user] += count

# Step 2: Convert the dictionary into a Pandas DataFrame for easy viewing
reaction_summary = pd.DataFrame(
    list(user_reaction_counts.items()), 
    columns=['User', 'Total Reaction Count']
)

# Step 3: Sort the results by reaction count in descending order
reaction_summary = reaction_summary.sort_values(by='Total Reaction Count', ascending=False)

# Display the results
print("Reaction counts by user:")
display(reaction_summary)

Reaction counts by user:


,User,Total Reaction Count
3,Dmytro Bibikov,133
4,Olena Toptunenko,101
2,Pavel,77
1,Olya Baranets,70
0,Карина Коник,58
7,🏡Oleksandr Toptunenko,22
5,Sashko Kucherov,20
6,Natalie Les,16
9,Anton Bludov,9
10,Daria Panchenko,9


In [9]:
from collections import defaultdict
import pandas as pd

# Step 1: Initialize a nested dictionary to track emojis per user
user_emoji_counts = defaultdict(lambda: defaultdict(int))

# Step 2: Iterate through the messages and extract reactions
for _, row in filtered_messages_with_reactions.iterrows():
    reactions = row.get('reactions')
    if isinstance(reactions, list):  # Ensure reactions exist and are a list
        for reaction in reactions:
            emoji = reaction.get('emoji')  # Emoji used
            count = reaction.get('count', 0)  # Reaction count
            
            # Check 'recent' reactions for users
            if 'recent' in reaction and isinstance(reaction['recent'], list):
                for recent_reaction in reaction['recent']:
                    user = recent_reaction.get('from')  # User reacting
                    if user and emoji:
                        user_emoji_counts[user][emoji] += count

# Step 3: Flatten the nested dictionary into a DataFrame
user_emoji_flat = []

for user, emojis in user_emoji_counts.items():
    for emoji, count in emojis.items():
        user_emoji_flat.append({'User': user, 'Emoji': emoji, 'Count': count})

# Convert to DataFrame
user_emoji_df = pd.DataFrame(user_emoji_flat)

# Step 4: Sort and display results
user_emoji_df = user_emoji_df.sort_values(by=['User', 'Count'], ascending=[True, False])

print("Emoji Reaction Count Per User:")
display(user_emoji_df)

Emoji Reaction Count Per User:


,User,Emoji,Count
56,Anton Bludov,😁,8
57,Anton Bludov,👍,1
58,Daria Panchenko,😁,6
60,Daria Panchenko,😱,2
59,Daria Panchenko,👍,1
...,...,...,...
49,🏡Oleksandr Toptunenko,😁,11
52,🏡Oleksandr Toptunenko,🔥,4
51,🏡Oleksandr Toptunenko,💯,3
53,🏡Oleksandr Toptunenko,😱,3


In [16]:
pivot_table = user_emoji_df.pivot_table(
    index='User',      # Users as rows
    columns='Emoji',   # Emojis as columns
    values='Count',    # Reaction count as values
    aggfunc='sum',     # Sum up counts
    fill_value=0       # Replace NaN with 0 for missing emoji counts
)

In [17]:
import altair as alt

# Example pivot_table-like data (convert your pivot_table to this format)
data = pivot_table.reset_index().melt(id_vars='User', var_name='Emoji', value_name='Reaction Count')

# Create heatmap
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('Emoji:N', title="Emojis"),  # Emojis as columns
    y=alt.Y('User:N', title="Users"),   # Users as rows
    color=alt.Color('Reaction Count:Q', scale=alt.Scale(scheme='blues'), title="Reaction Count"),  # Heatmap color scale
    tooltip=['User', 'Emoji', 'Reaction Count'],  # Hover tooltips
).properties(
    title="Emoji Reaction Counts Per User",
    width=500,  # Chart width
    height=300  # Chart height
)

heatmap.show()

alt.Chart(...)